In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, collections
from tf.fabric import Fabric

In [3]:
SBL = 'greek/sblgnt'
TF = Fabric( modules=SBL )

This is Text-Fabric 2.0.0
Api reference : https://github.com/ETCBC/text-fabric/wiki/Api
Tutorial      : https://github.com/ETCBC/text-fabric/blob/master/docs/tutorial.ipynb
Data sources  : https://github.com/ETCBC/text-fabric-data
Data docs     : https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/0_overview.html
Shebanq docs  : https://shebanq.ancient-data.org/text
Slack team    : https://shebanq.slack.com/signup
Questions? Ask shebanq@ancient-data.org for an invite to Slack
31 features found and 0 ignored


In [8]:
api = TF.load('''
    Cat
    function
    child
''')
api.makeAvailableIn(globals())

  0.00s loading features ...
   |     0.00s Feature overview: 28 nodes; 2 edges; 1 configs; 7 computeds
  0.95s All features loaded/computed - for details use loadLog()


In [9]:
cats = collections.Counter()
for n in F.otype.s('wordx'):
    cats[F.Cat.v(n)] += 1
cats

Counter({'adj': 236,
         'adv': 138,
         'intj': 298,
         'prep': 141,
         'pron': 2,
         'ptcl': 64})